In [1]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

In [2]:
from numpy import load
import numpy as  np
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
from keras_facenet import FaceNet
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

## Localizing & Extracting Faces

In [3]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [4]:
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN

In [5]:
#extract a single face from a given photograph
def extract_face(filename, required_size = (160, 160)):
    #load image from file
    image = Image.open(filename)
    #convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    # bug fix
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

#load images and extract faces for all images in a directory
def load_faces(directory):
    faces = list()
    filenames_list = list() #NJ: A list to store all the images file names
    
#     #enumerate files
#     for filename in listdir(directory):
#         # path
#         path = directory + filename
        # get face
    
    path = directory
    face = extract_face(path)
        # store
    faces.append(face)
    filenames_list.append(path) 
        
    return faces, filenames_list

#load & extract faces for a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
    X, y, path_list = list(), list(), list()
    
    # enumerate folders, one per class
    for subdir in listdir(directory):
        #path
        path = directory + '/' + subdir
        # skip any files that might be in the dir
        
#         if not isdir(path):
#             continue
            
        # load all faces in the subdirectory
        faces, filenames_list = load_faces(path)
        
        # create labels
        labels = [subdir for _ in range(len(faces))]
        
        # summarize progress
        print ('>loaded %d example for class %s' % (len(faces), subdir))
        # store
        X.extend(faces)
        y.extend(labels)
        path_list.extend(filenames_list) # to get the actual image names
        
    return asarray(X), asarray(y), asarray(path_list)

## Faces Extraction from dataset

In [6]:
# load train dataset
dataset_train_path = 'dataset'

faces_dataset = load_dataset(dataset_train_path)

>loaded 1 example for class 1.PNG
>loaded 1 example for class 10.PNG
>loaded 1 example for class 11.PNG
>loaded 1 example for class 12.PNG
>loaded 1 example for class 13.PNG
>loaded 1 example for class 14.PNG
>loaded 1 example for class 15.PNG
>loaded 1 example for class 16.PNG
>loaded 1 example for class 17.PNG
>loaded 1 example for class 18.PNG
>loaded 1 example for class 19.PNG
>loaded 1 example for class 2.PNG
>loaded 1 example for class 20.PNG
>loaded 1 example for class 21.PNG
>loaded 1 example for class 22.PNG
>loaded 1 example for class 23.PNG
>loaded 1 example for class 24.PNG
>loaded 1 example for class 25.PNG
>loaded 1 example for class 26.PNG
>loaded 1 example for class 27.PNG
>loaded 1 example for class 28.PNG
>loaded 1 example for class 29.PNG
>loaded 1 example for class 3.PNG
>loaded 1 example for class 30.PNG
>loaded 1 example for class 4.PNG
>loaded 1 example for class 5.PNG
>loaded 1 example for class 6.PNG
>loaded 1 example for class 7.PNG
>loaded 1 example for class

In [7]:
listdir(dataset_train_path)

['1.PNG',
 '10.PNG',
 '11.PNG',
 '12.PNG',
 '13.PNG',
 '14.PNG',
 '15.PNG',
 '16.PNG',
 '17.PNG',
 '18.PNG',
 '19.PNG',
 '2.PNG',
 '20.PNG',
 '21.PNG',
 '22.PNG',
 '23.PNG',
 '24.PNG',
 '25.PNG',
 '26.PNG',
 '27.PNG',
 '28.PNG',
 '29.PNG',
 '3.PNG',
 '30.PNG',
 '4.PNG',
 '5.PNG',
 '6.PNG',
 '7.PNG',
 '8.PNG',
 '9.PNG']

In [8]:
savez_compressed('./faces-train_dataset.npz', faces_dataset[0], faces_dataset[1], faces_dataset[2])

In [9]:
# load the face dataset
data = load('./faces-train_dataset.npz')

trainX, trainy, train_images_path = data['arr_0'], data['arr_1'], data['arr_2'] #, data['arr_3']
print('Loaded Train dataset: ', trainX.shape, trainy.shape, train_images_path.shape)

Loaded Train dataset:  (30, 160, 160, 3) (30,) (30,)


## Feature Extraction for the Faces extracted from Dataset

In [10]:
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
from keras_facenet import FaceNet

embedder = FaceNet()

In [11]:
features_dataset = embedder.embeddings(trainX)

In [12]:
# save arrays to one file in compressed format
savez_compressed('./faces-train_embeddings.npz', features_dataset, trainy, train_images_path)

## Finding Duplicate Images

In [13]:
#load dataset
data = load('./faces-train_embeddings.npz')
trainX_features, trainy, train_images_path = data['arr_0'], data['arr_1'], data['arr_2'] #, data['arr_3']

In [14]:
def get_duplicate_images_df(feature, matching_cutoff_threshold = 0.8):
    img_dup = pd.DataFrame([],columns=['image_id','Image_label','Image_path','Duplicate_image','Duplicate_path','similarity_score'])
    for id1,feature1 in enumerate(zip(feature['arr_0'], feature['arr_1'], feature['arr_2'])):
        for id2,feature2 in enumerate(zip(feature['arr_0'], feature['arr_1'], feature['arr_2'])):
            if id2>id1:
                image_1,label_1,path_1 = feature1[0],feature1[1],feature1[2]
                image_2,label_2,path_2 = feature2[0],feature2[1],feature2[2]

                similarity_score = cosine_similarity(image_1.reshape(1, -1),
                                                     image_2.reshape(1, -1))
                if similarity_score > matching_cutoff_threshold:
                    img_dup.loc[len(img_dup)]= [id1,label_1,path_1,label_2,path_2,similarity_score[0][0]]
    return img_dup

In [15]:
df_dup = get_duplicate_images_df(data)
df_dup

,image_id,Image_label,Image_path,Duplicate_image,Duplicate_path,similarity_score
0,6,15.PNG,dataset/15.PNG,30.PNG,dataset/30.PNG,0.883377


In [16]:
#df_dup.to_csv('Duplicate_images.csv', index = None, header=True)